In [1]:
using Pipe
using DataFrames
# using StatsPlots
using CSV
# using RollingFunctions
using Query
# using GLMakie

using JSON
using DataFrames
using Dates
using Colors
# using ColorSchemes
using Indicators

import Colors
using Plots
using Interact
# using JuliaDB
# using Wavelets,WaveletsExt
# using LinearAlgebra
# import Pkg; Pkg.add("ShiftedArrays")
# using ShiftedArrays
include("utils.jl")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-10866643820967124776\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-11019370979032038234\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-10866643820967124776\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

calc_wavelet (generic function with 1 method)

In [2]:
json_data_folder = "../user_data/data/binance"
json_filename = "BTC_USDT-1m.json"
json_filepath = "$json_data_folder/$json_filename"
f = open(json_filepath, "r")
json_data = JSON.parse(f)
close(f)

In [3]:
dfa = DataFrame(unix_time = Int64[], open=Float64[], high=Float64[], low=Float64[], close=Float64[], volume=Float64[])
for r in json_data push!(dfa, r) end
dfa.unix_time .= dfa.unix_time ./1000
# dfa.time .= unix2datetime.(dfa.unix_time)

insertcols!(dfa, :time => unix2datetime.(dfa.unix_time))
nrow(dfa)

129177

In [4]:
start_date = DateTime(2021, 4, 1)
end_date = DateTime(2021, 5, 27)
vdf = dfa[(dfa.time .>= start_date) .& (dfa.time .<= end_date) , :]
ut, tt, hh, ll, cc = vdf.unix_time, vdf.time, vdf.high, vdf.low, vdf.close
# h14max = [fill(vdf[1,:high],8-1)...,rollmax(vdf.high, 8)...]
#  length(h14max), length(vdf.close)
# hlc = [hh ll cc]
length(tt)

80207

In [5]:
ar = cc
rsi14 = rsi(ar,n=14)
rsi26 = rsi(ar,n=26)
# ema = Indicators.ema
# ema26 = ema(ar,n=26)
ema50 = ema(ar,n=330)
ema100 = ema(ar,n=100)
ema200 = ema(ar,n=200)
# ema400 = ema(ar,n=400)
# sma = Indicators.sma
# sma26 = sma(ar,n=26)
# sma50 = sma(ar,n=50)
# sma100 = sma(ar,n=100)
# sma200 = sma(ar,n=200)
# sma400 = sma(ar,n=400)
# sma800 = sma(ar,n=800)
# ppsar = psar(ar)
ppsar = psar([hh ll])
1
# ar_emas = Dict([i => ema(ar, n=2^i) for i in 4:9])

1

In [12]:
ar = cc

capital = 10000.
risk_percent = 0.01
qfee = 0.001

dfo = DataFrame(ut = Int64[], 
        order = Symbol[],
        reason = Symbol[],
        price = Float64[],
        stop = Float64[],
        qty = Float64[],
        pnl = Float64[],
        capital = Float64[]
    )

push!(dfo, (ut = ut[1], order = :long, reason = :toolong, price = 0, stop = 0, qty = 0, pnl = 0, capital = capital))

in_long_pos = false
qlbuysig = 0.98
qlsellsig = 1.01
qlstopsig = 0.99

ar = cc
# arsma100 = Float64[]
# arema100 = Float64[]

for i in 1:length(ar)
    
    ord = last(dfo)
    
    if !in_long_pos
        if rsi26[i] <= 28 # && ema50[i] > (ema100[i] .* 0.997) && ema100[i] >= (ema200[i] .* 1.001)
            risk_size = capital * risk_percent
            stop_price = ar[i] * 0.99 * (1-qfee)
            
            buy_price = ar[i]*(1+qfee)
            buyq = risk_size / (buy_price - stop_price)
            
            push!(dfo, (ut = ut[i], order = :long, reason = :enter, price = buy_price, qty = buyq,
                    stop = stop_price, pnl = 0, capital = capital))
            
            in_long_pos = true
        end
    else
        if ar[i] >= ord.price * qlsellsig  # target         
            sell_price = ar[i]*(1-qfee)
            profit_loss = ord.qty * (sell_price - ord.price)
            capital = capital + profit_loss
            push!(dfo, (ut = ut[i], order = :long, reason = :target, price = sell_price,
                    stop = 0, qty = ord.qty, pnl = profit_loss, capital = capital))
            
            in_long_pos = false
#             println("TARGET:\t$profit_loss, ")
            
        elseif ar[i] <= ord.stop # SL
            sell_price = ar[i]*(1-qfee)
            profit_loss = ord.qty * (sell_price - ord.price)
            capital = capital + profit_loss
            
            push!(dfo, (ut = ut[i], order = :long, reason = :stoploss, price = sell_price, stop = 0, 
                    qty = ord.qty, pnl = profit_loss, capital = capital))
            in_long_pos = false
#             println("STOPLOSS:\t$profit_loss,")
        end
        
    end
#     println(capital)
    
end

In [ ]:
last(dfo)

In [ ]:
dfo

In [ ]:
ar = cc
ar_len = length(ar)
plot_box = Observable{Any}(dom"div"())


window_size = 60 * 8 *3

num_windows = Int64(trunc(ar_len/window_size*4))

window_no_slider = slider(1:num_windows; label="start", default=7)

window_start = Interact.@map max(1, Int(&window_no_slider * window_size / 4))
window_end = Interact.@map min(ar_len, &window_start+window_size-1)

rng = Interact.@map &window_start: &window_end

put = Interact.@map ut[&rng]
par = Interact.@map ar[&rng]
prsi14 = Interact.@map rsi14[&rng]
prsi26 = Interact.@map rsi26[&rng]
pema50 = Interact.@map ema50[&rng]
pema100 = Interact.@map ema100[&rng]
pema200 = Interact.@map ema200[&rng]
pppsar = Interact.@map ppsar[&rng]

wdfo = Interact.@map dfo[(dfo.ut .>= first(&put)) .& (dfo.ut .<= last(&put)), :]

# buys = Interact.@map filter(row -> row.reason .== :long, &wdfo) 
# targets = Interact.@map filter(row -> row.reason .== :target, &wdfo) 
# stoplosses = Interact.@map filter(row -> row.reason .== :stoploss, &wdfo) 

function draw_it_all(ut, ar, ema50,ema100,ema200, rsi14,rsi26, ppsar, dfo)
    
#     wtdo = dfo[(dfo.ut .>= ut[window_start]) .& (dfo.ut .<= ut[window_end]), :]
    
    buys = dfo[dfo.reason .== :enter,:]
    targets = dfo[(dfo.reason .== :target),:]
    stoplosses = dfo[(dfo.reason .== :stoploss),:]
    
    f1 = plot(ut, ar) 
    plot!(ut,ema50)
    plot!(ut,ema100)
    plot!(ut,ema200)
    scatter!(ut,ppsar)
    
    nrow(buys) > 0 && vline!(buys.ut, color=:blue)
    nrow(targets) > 0 && vline!(targets.ut, color=:green)
    nrow(stoplosses) > 0 && vline!(stoplosses.ut, color=:red)
    nrow(buys) > 0 && scatter!(buys.ut, buys.price * qlsellsig, color=:green)
    nrow(buys) > 0 && scatter!(buys.ut, buys.price * qlstopsig, color=:red)

#     vline!(ut[Int(length(ut)/2)])
    f2 = plot(ut,rsi26, linewidth=1, color=:blue)
#     plot!(ut,rsi26)
    hline!([30,70])
    plt = plot(f1, f2, layout=@layout[a{0.6h}; b{0.25h};], legend=nothing, size=(1200,400))
    dom"div"(vbox(plt,dfo))
end

map!(draw_it_all, plot_box, put, par, pema50,pema100,pema200, prsi14, prsi26,pppsar, wdfo)
ui = dom"div"(window_no_slider,plot_box)


In [ ]:
nrow(dfo)

In [ ]:
last(dfo)

In [ ]:
maema_1_ar

In [ ]:
wnd[:window_size] 

In [ ]:
wave_slider[]

In [ ]:
i = 1000
i - max(1,i - 512)

In [ ]:
wt = wavelet(WT.Coiflet{4}(), WT.Filter, WT.Periodic)

In [ ]:
2^8

In [ ]:
length(ar)